In [1]:
import matplotlib.pyplot as plt
import numpy as np
import scipy
from copy import deepcopy
import random
import numba as nb
import time
import warnings
warnings.filterwarnings("ignore")

def tensor(A, B):
    m = len(A)*len(B)
    n = len(A[0])*len(B[0])

    prod = []

    for i in range(m):
        prod.append([])
        for j in range(n):
            ax = int(i / len(B))
            ay = int(j / len(B[0]))
            bx = int(i % len(B))
            by = int(j % len(B[0]))
            prod[i].append(A[ax][ay]*B[bx][by])

    return np.array(prod)

def tensor_list(ops):
    prod = ops[-1]
    for i in range(len(ops)-2, -1, -1):
        prod = tensor(ops[i], prod)
    return np.array(prod)

def pauli(pos, dir, N):
    factors = [[[1, 0], [0, 1]]]*N
    if dir == 0 or dir == 'x':
        factors[pos] = [[0, 1], [1, 0]]
    elif dir == 1 or dir == 'y': # y
        factors[pos] = [[0, -1j], [1j, 0]]
    elif dir == 2 or dir == 'z': # z
        factors[pos] = [[1, 0], [0, -1]]

    return tensor_list(factors)

def on_site_detuning(pos, N, V_nn, V_nnn):
    sum = 0
    for j in range(N):
        if np.abs(pos - j) == 1:
            sum += V_nn
        elif np.abs(pos - j) == 2:
            sum += V_nnn

    return -0.5 * sum

def interaction_strength(pos1, pos2, V_nn, V_nnn):
    if np.abs(pos1 - pos2) == 1:
        return V_nn
    if np.abs(pos1 - pos2) == 2:
        return V_nnn
    else:
        return 0

In [9]:
# construct Hamiltonian
rabi_f = 2*np.pi*6.4*(10**6) # rabi frequency for Rydberg to ground state cycles (Hz)
V_nn = 2*np.pi*60*(10**6) # nearest neighbors interaction strength (Hz) (ref page 180, strongly interacting pairs)
V_nnn = 2*np.pi*2.3*(10**6) # next nearest neighbors interaction strength (Hz)

def hamiltonian(N, rabi_f, delta, V_nn, V_nnn):
    H = 0.5 * rabi_f * sum([pauli(i, 'x', N) for i in range(N)])

    print(0.5 * rabi_f)

    H -= 0.5 * sum([(delta + on_site_detuning(i, N, V_nn, V_nnn)) * pauli(i, 'z', N) for i in range(N)])
    print()
    for i in range(N):
        print(-0.5 * (delta + on_site_detuning(i, N, V_nn, V_nnn)))

    print()
    for i in range(N):
        for j in range(N):
            print(i, j, 0.125*interaction_strength(i, j, V_nn, V_nnn))

    interaction_term = sum([sum([interaction_strength(i, j, V_nn, V_nnn) * pauli(i, 'z', N) @ pauli(j, 'z', N) for j in range(N)]) for i in range(N)])
    H += 0.125 * interaction_term
    return H

In [10]:
hamiltonian(3, rabi_f, 0, V_nn, V_nnn)

20106192.982974675

97860611.15932205
188495559.21538758
97860611.15932205

0 0 0.0
0 1 47123889.803846896
0 2 1806415.775814131
1 0 47123889.803846896
1 1 0.0
1 2 47123889.803846896
2 0 1806415.775814131
2 1 47123889.803846896
2 2 0.0


array([[ 5.76325172e+08,  2.01061930e+07,  2.01061930e+07,
         0.00000000e+00,  2.01061930e+07,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00],
       [ 2.01061930e+07,  1.84882728e+08,  0.00000000e+00,
         2.01061930e+07,  0.00000000e+00,  2.01061930e+07,
         0.00000000e+00,  0.00000000e+00],
       [ 2.01061930e+07,  0.00000000e+00, -1.77657065e+08,
         2.01061930e+07,  0.00000000e+00,  0.00000000e+00,
         2.01061930e+07,  0.00000000e+00],
       [ 0.00000000e+00,  2.01061930e+07,  2.01061930e+07,
        -1.92108391e+08,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  2.01061930e+07],
       [ 2.01061930e+07,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  1.84882728e+08,  2.01061930e+07,
         2.01061930e+07,  0.00000000e+00],
       [ 0.00000000e+00,  2.01061930e+07,  0.00000000e+00,
         0.00000000e+00,  2.01061930e+07, -1.92108391e+08,
         0.00000000e+00,  2.01061930e+07],
       [ 0.00000000e+00,  0.000000